3→2の全結合層fcを定義(重みは最初ランダム)
入力をxと置く
xをfcに入れて線形変換された値u1を
活性化関数(今回はRelu関数)F.reluに入れ非線形変換された値hにする
hを同様にu2、そして予測値yへと変換する。
yと目標値tを損失関数F.mse_lossに入れて誤差を計算し
逆伝播でモデルを学習させていく


In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch
import torch.nn as nn


In [98]:
#fullly connected layer
torch.manual_seed(0)
fc1 = nn.Linear(3,2)
fc1

Linear(in_features=3, out_features=2, bias=True)

In [99]:
fc1.weight

Parameter containing:
tensor([[-0.0043,  0.3097, -0.4752],
        [-0.4249, -0.2224,  0.1548]], requires_grad=True)

In [100]:
fc1.bias

Parameter containing:
tensor([-0.0114,  0.4578], requires_grad=True)

In [101]:
torch.__version__

'2.5.1+cpu'

In [102]:
x = torch.tensor([[1., 2., 3.]])
type(x), x.dtype

(torch.Tensor, torch.float32)

In [103]:
u1 = fc1(x)
u1 

tensor([[-0.8219,  0.0526]], grad_fn=<AddmmBackward0>)

In [104]:
import torch.nn.functional as F


In [105]:
h = F.relu(u1)
h

tensor([[0.0000, 0.0526]], grad_fn=<ReluBackward0>)

In [106]:
torch.manual_seed(0)
fc2 = nn.Linear(2,1)
u2 = fc2(u1)


In [107]:
y = F.relu(u2)
y

tensor([[0.]], grad_fn=<ReluBackward0>)

In [108]:
t = torch.tensor([[1.]])
t

tensor([[1.]])

In [109]:
F.mse_loss(y, t)

tensor(1., grad_fn=<MseLossBackward0>)